In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from tqdm.notebook import tqdm
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models

from sklearn.model_selection import GroupKFold
import efficientnet.tfkeras

In [2]:
class CFG:
   debug=True
   dataset_dir="../input/ranzcr-clip-catheter-line-classification/"
   models_dir="./models/" if debug else "../input/ranzcr-chibamed/"
   batch_size=4 if debug else 128
   input_shape=(600,600)

   target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
      'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']
   n_folds=5

In [3]:
test=pd.read_csv(f"{CFG.dataset_dir}train.csv") if CFG.debug else pd.read_csv(f"{CFG.dataset_dir}sample_submission.csv")

In [11]:
effnet=models.load_model(f"{CFG.models_dir}xhlulu/model.h5")
layer_name="global_average_pooling2d"
hidden_layer_model=models.Model(inputs=effnet.input,outputs=effnet.get_layer(layer_name).output)
hidden_layer_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7_input (Input [(None, 600, 600, 3)]     0         
_________________________________________________________________
efficientnet-b7 (Functional) (None, 19, 19, 2560)      64097680  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
Total params: 64,097,680
Trainable params: 63,786,960
Non-trainable params: 310,720
_________________________________________________________________


In [12]:
def preprocess(uid):
    if CFG.debug:
        file_bytes=tf.io.read_file(f"{CFG.dataset_dir}train/"+uid+".jpg") # f"{}"形式ではTensorの型変換が働かない
    else:
        file_bytes=tf.io.read_file(f"{CFG.dataset_dir}test/"+uid+".jpg")
    image=tf.io.decode_jpeg(file_bytes,channels=3)
    image=tf.cast(image,tf.float32)
    image=tf.image.resize(image,CFG.input_shape)
    image/=255
    return uid,image

def decode_string(tensor):
    return [str_bytes.decode() for str_bytes in tensor.numpy()]

AUTO=tf.data.experimental.AUTOTUNE
dset=tf.data.Dataset.from_tensor_slices(test["StudyInstanceUID"])
dset=dset.map(preprocess,num_parallel_calls=AUTO)
dset=dset.batch(CFG.batch_size).prefetch(AUTO)

In [13]:
ndarray_dict={}
for uids,imgs in tqdm(dset):
    preds=hidden_layer_model.predict(imgs)
    uids_decoded=decode_string(uids)
    for uid,pred in zip(uids_decoded,preds):
        ndarray_dict[uid]=pred

    if CFG.debug:
        break

  0%|          | 0/7521 [00:00<?, ?it/s]

In [15]:
if CFG.debug:
    features_list=[ndarray_dict[uid] for uid in ndarray_dict.keys()]
else:
    features_list=[ndarray_dict[uid] for uid in tqdm(test["StudyInstanceUID"])]
features=np.array(features_list)

In [18]:
def create_model(fold:int):
    dense_dir=f"{CFG.models_dir}effnet_best_dense/"
    with open(f"{dense_dir}model.json","rt") as f:
        model_json=f.read()
    dense_model=models.model_from_json(model_json)
    dense_model.load_weights(f"{dense_dir}ckpt_{fold}")
    return dense_model

dense_model_list=[create_model(n) for n in range(CFG.n_folds)]

In [19]:
dense_pred_list=[pd.DataFrame(dense_model_list[n].predict(features),columns=CFG.target_cols) for n in range(CFG.n_folds)]

submission=test.copy()

for col_name in CFG.target_cols:
    pred_col=pd.DataFrame()
    for n,pred in enumerate(dense_pred_list):
        pred_col[f"fold_{n}"]=pred[col_name]
    submission[col_name]=pred_col.mean(axis=1)
display(submission)

if not CFG.debug:
    submission.to_csv("submission.csv",index=False)

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0.001258,0.000487,0.000332,0.011385,0.137660,0.008421,0.674000,0.222881,0.128203,0.036015,0.000304,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0.005584,0.063125,0.963192,0.226726,0.044443,0.899896,0.020829,0.068652,0.041213,0.980235,0.011262,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0.000309,0.000526,0.000213,0.000352,0.000222,0.000046,0.000722,0.004723,0.822312,0.263706,0.000123,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0.000848,0.002815,0.002329,0.002313,0.001520,0.001542,0.005733,0.449443,0.548132,0.042099,0.000744,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207685cd1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5b5b9ac30
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7192404d8
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d4d1b066d
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01a6602b8
